In [2]:
import os
from google.colab import userdata
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

In [3]:
%%capture
#hides output

major_version, minor_version = torch.cuda.get_device_capability()

!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"


In [4]:
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)

    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
    print("used A100 GPU")
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)

    !pip install --no-deps xformers trl peft accelerate bitsandbytes
    print("used t4 GPU")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.2/218.2 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.1 MB/s eta 0:00:00
used t4 GPU


In [13]:
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [6]:
# # 4bit pre quantized models UNSLOTH support - 4x faster downloading!
# fourbit_models = [
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/llama-2-7b-bnb-4bit",
#     "unsloth/llama-2-13b-bnb-4bit",
#     "unsloth/codellama-34b-bnb-4bit",
#     "unsloth/tinyllama-bnb-4bit",
# ] # Go to https://huggingface.co/unsloth for more 4-bit models!

In [7]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit", # We can Choose ANY! hugging face model eg teknium/OpenHermes-2.5-Mistral-7B
                                                #choosing "unsloth/mistral-7b-bnb-4bit" for 4x fast download
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
    dtype = dtype, #computation datatype
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.24. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [11]:
EOS_TOKEN = tokenizer.eos_token
print(EOS_TOKEN)

</s>


In [8]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.3 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [9]:
from datasets import load_dataset
dataset_name="mirajbhandari/mistral_dataset_for_tuning"
dataset = load_dataset(dataset_name, split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [14]:
training_arguments = TrainingArguments(
    output_dir="outputs",
    num_train_epochs= 50,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    optim="adamw_8bit",
    save_steps=0,
    logging_steps= 1,
    learning_rate= 2e-4,
    weight_decay= 0.01,
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard"
    save_strategy="steps",  # Save based on steps or epoch
    save_total_limit=1,  # Save only the best model
)


In [17]:

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="0",
    max_seq_length=max_seq_length,
    args=training_arguments,
    packing=False,
)

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

In [18]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 24 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 300
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
1,3.486700
2,3.388800
3,3.009000
4,2.862400
5,2.500300
6,2.108600
7,2.251100
8,1.768700
9,1.186400
10,1.023700


In [22]:
user_question = "Does TechVibe provide software updates and security patches for its mobile devices?"

que =[{"role": "user", "content": user_question}]

# Format user input question
formatted_question=tokenizer.apply_chat_template(que, tokenize=False)

# Send formatted question to the model for prediction
device = "cuda:0"
inputs = tokenizer(formatted_question, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20,use_cache = True)
predicted_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print predicted output
print(predicted_output)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] Does TechVibe provide software updates and security patches for its mobile devices? [/INST] Yes, TechVibe regularly releases software updates and security patches to ensure optimal performance and user security.


In [23]:
#save lora adapters
model.save_pretrained("lora_model")
tokenizer.save_pretrained("lora_model")

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.model',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

In [25]:
model.save_pretrained_merged("model-16bit", tokenizer, save_method = "merged_16bit")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which will take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.74 out of 12.67 RAM for saving.


100%|██████████| 32/32 [00:55<00:00,  1.74s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving model-16bit/pytorch_model-00001-of-00003.bin...
Unsloth: Saving model-16bit/pytorch_model-00002-of-00003.bin...
Unsloth: Saving model-16bit/pytorch_model-00003-of-00003.bin...
Done.


In [26]:
model.push_to_hub_merged("mirajbhandari/unsloth_mistral_16bit", tokenizer, save_method = "merged_16bit")

Unsloth: You are pushing to hub, but you passed your HF username = mirajbhandari.
We shall truncate mirajbhandari/unsloth_mistral_16bit to unsloth_mistral_16bit


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 5.51 out of 12.67 RAM for saving.


100%|██████████| 32/32 [01:06<00:00,  2.09s/it]


Unsloth: Saving tokenizer...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

...

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/mirajbhandari/unsloth_mistral_16bit


In [28]:
# for inference if we want to inference from lora adapters only

In [24]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

user_question = "What does techvibe produce?"

que =[{"role": "user", "content": user_question}]

# Format user input question
formatted_question=tokenizer.apply_chat_template(que, tokenize=False)

# Send formatted question to the model for prediction
device = "cuda:0"
inputs = tokenizer(formatted_question, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=200,use_cache = True)
predicted_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print predicted output
print(predicted_output)

==((====))==  Unsloth: Fast Mistral patching release 2024.3
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.24. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:155: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[INST] What does techvibe produce? [/INST] TechVibe is a leading manufacturer of mobile devices and accessories.


In [27]:
# from peft import AutoPeftModelForCausalLM
# from transformers import AutoTokenizer,BitsAndBytesConfig

# quantization_config = BitsAndBytesConfig(load_in_4bit=True)

# model = AutoPeftModelForCausalLM.from_pretrained(
#         "lora_model", # YOUR MODEL YOU USED FOR TRAINING
#          quantization_config=quantization_config
#     )
# tokenizer = AutoTokenizer.from_pretrained("lora_model")

# user_question = "What does techvibe produce?"

# que =[{"role": "user", "content": user_question}]

# # Format user input question
# formatted_question=tokenizer.apply_chat_template(que, tokenize=False)

# # Send formatted question to the model for prediction
# device = "cuda:0"
# inputs = tokenizer(formatted_question, return_tensors="pt").to(device)

# outputs = model.generate(**inputs, max_new_tokens=200,use_cache = True)
# predicted_output = tokenizer.decode(outputs[0], skip_special_tokens=True)

# # Print predicted output
# print(predicted_output)